In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.python.client import device_lib 
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

tf.random.set_seed(42)

# Global variable specifying the project directory. This variable helps make the code easier
# to run in multiple locations. (For example on local pc vs google colab enviornments for 
# various team members). 
global PROJECT_DIRECTORY
PROJECT_DIRECTORY = "/content/drive/MyDrive/Programming/Colab Notebooks/General_Assembly/Car/"

# Folder paths for my own computer
#train_directory = "./data/organized/train/"
#val_directory = "./data/organized/val/"
#test_directory = "./data/organized/test/"

# Folder paths when using Google Colab and after copying the data to the folder where the colab notebook runs.
train_dir = "./organized/train"
val_dir = "./organized/val"
test_dir = "./organized/test"

# Allow pandas to display more than 50 rows when calling .head()
pd.set_option('display.max_rows', 200)

**A note on training models with large datasets on Google Colab**

My project has been set up such that image files are read in one batch at a time as they are needed for training the model. This is used so we can avoid reading the entire dataset into RAM at once, which depending on the system may be simply inefficient or even impossible. This process presents a separate set of challenges when trying to implement it on Google Colaboratory. Specifically, Google Colab is very inefficient when reading large files from Google Drive. This means that model training times are significantly increased, to the point where the code is spending most of its time reading files and a relatively small amount of time actually utilizing the valuable GPUs. 

The solution to the issue described above is to copy the dataset from drive to the folder where colab is actually running prior to model training. This significantly increases the speed at which image files can be read in, and therefore also significantly increase model training times. The code to perform this copy and paste is shown in the following two cells, and is also described in this article: 

https://medium.datadriveninvestor.com/speed-up-your-image-training-on-google-colab-dc95ea1491cf

In [ ]:
# Path to where I stored the Stanford Cars dataset images on google drive.
data_path = "/content/drive/MyDrive/Programming/Colab Notebooks/General_Assembly/Car/data/organized/"

# This command takes the folder containing the stanford cars dataset images and creates a new zipped version of the folder.
!(cd '{data_path}' && zip -r -q organized_data.zip organized)

In [ ]:
# Path to to the zipped stanford cars dataset images.
zip_path = "/content/drive/MyDrive/Programming/Colab Notebooks/General_Assembly/Car/data/organized_data.zip"

# Copy the zip folder containing the images to the directory where google colab is running.
!cp '{zip_path}' .

# Unzip the images in the folder that google colab uses.
!unzip -q organized_data.zip

# Remove the zipped version of the folder.
!rm organized_data.zip

In [ ]:
# ===============================================================================================================
# This function creates training, validation and test datasets using the file structure created in the
# 01_create_train_val_test_directories notebook. 
# ===============================================================================================================
def create_tensorflow_datasets(image_size, train_directory, val_directory, test_directory, batch_size=32):
    
    train_dataset = image_dataset_from_directory(directory = train_directory,
                                                 labels='inferred',
                                                 label_mode = 'int',
                                                 image_size=image_size,
                                                 batch_size=batch_size,
                                                 smart_resize=True)

    val_dataset = image_dataset_from_directory(directory = val_directory,
                                               labels='inferred',
                                               label_mode = 'int',
                                               image_size=image_size,
                                               batch_size=batch_size,
                                               smart_resize=True)

    test_dataset = image_dataset_from_directory(directory = test_directory,
                                                labels = "inferred",
                                                label_mode = "int",
                                                image_size=image_size,
                                                batch_size=batch_size,
                                                smart_resize=True)
    
    return train_dataset, val_dataset, test_dataset

In [ ]:
# ===============================================================================================================
# Helper function to inspect the contents of a tensorflow dataset.
# ===============================================================================================================
def inspect_tf_dataset(tf_dataset):
    for batch, labels in tf_dataset:
        print(f"Batch Shape: {batch.shape}")
        print(f"Labels Shape: {labels.shape}\n")
        print(f"len(Batch):\n {len(batch)}\n")
        print(f"len(Labels):\n {len(labels)}\n")
        print(f"type Batch:\n {batch.dtype}\n")
        print(f"type Labels:\n {labels.dtype}\n")
        
        
        for batch_num in range(1):
            print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
            print(f"Batch Item Number: {batch_num}")
            print(f"Batch[{batch_num}].shape:\n {batch[batch_num].shape}\n")
            print(f"Labels[{batch_num}].shape:\n {labels[batch_num].shape}\n")
            print(f"type Batch[{batch_num}].shape:\n {batch[batch_num].dtype}\n")
            print(f"type Labels[{batch_num}].shape:\n {labels[batch_num].dtype}\n")
            print(f"Batch[{batch_num}]:\n {batch[batch_num]}\n")
            print(f"Labels[{batch_num}]:\n {labels[batch_num]}\n")
            print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
        break

In [ ]:
train_dataset, val_dataset, test_dataset = create_tensorflow_datasets(image_size=(520, 520),
                                                                      train_directory=train_dir,
                                                                      val_directory=val_dir,
                                                                      test_directory=test_dir,
                                                                      batch_size=32)

Found 10520 files belonging to 196 classes.
Found 3234 files belonging to 196 classes.
Found 2431 files belonging to 196 classes.


In [ ]:
# ===============================================================================================================
# This function implements a custom learning rate schedule.
#
# The learning rate is decreased by a factor of 1.3 every 10 epochs, until a minimum allowed learning rate of
# 1e-6 is reached.
# ===============================================================================================================
def learning_rate_scheduler(epoch, lr):
    if epoch % 10 == 0 and epoch != 0:
        
        updated_lr = max((lr / (1 + 0.30)), 0.000001)
        
        if updated_lr != lr: 
            print("/n====================================================")
            print("Updating the learning rate...")
            print(f"Previous LR: {lr}  Updated LR: {updated_lr}")
            print("====================================================\n")    
        return updated_lr 
    else:
        return lr

In [ ]:
# ===============================================================================================================
# This function creates a unique filename to save trained models. 
# ===============================================================================================================
def get_model_save_path(optimizer, lr, epochs, batch_size, model_name):
    
    global PROJECT_DIRECTORY

    # String
    optimizer_string = optimizer + str(lr).split('.')[1]
    time_stamp = time.strftime("%Y_%m_%d-%H_%M_%S")
    
    model_save_path = os.path.join(PROJECT_DIRECTORY, f"trained_models/convnet/{time_stamp}_{model_name}_E{epochs}_O{optimizer_string}_B{batch_size}.keras")
    
    return model_save_path

In [ ]:
# ===============================================================================================================
# Function to easily switch which optimizer is used for training.
# ===============================================================================================================
def get_optimizer(optimizer_name, lr):
    
    if optimizer_name == 'rmsprop':
        
        optimizer = tf.keras.optimizers.RMSProp(learning_rate = lr)
    
    elif optimizer_name == 'adam':
        
        optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
        
    return optimizer

In [ ]:
# ===============================================================================================================
# This function instantiates and compiles a convolutional neural network.
# 
# Keras preprocessing layers are used to implement random data augmentations throughout training.
# ===============================================================================================================
def build_convnet_classifier(input_shape, optimizer, learning_rate, metrics):
    
    # Get the specified optimizer.
    optimizer = get_optimizer(optimizer, learning_rate)
    
    # Create a layer that randomly augments the data prior to being input to the model.
    data_augmentation = keras.Sequential(
        [
            layers.experimental.preprocessing.RandomFlip("horizontal"),
            layers.experimental.preprocessing.RandomRotation(0.1),
            layers.experimental.preprocessing.RandomZoom(0.2),
        ]
    )
    
    # Input layer
    inputs = keras.Input(shape=input_shape)
    
    # Perform data augmentation
    x = data_augmentation(inputs)
    
    # Recale the pixel values from the 1-255 range to a 0-1 range
    x = layers.experimental.preprocessing.Rescaling(1./255)(x)
    
    x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    
    x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    
    x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    
    x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    
    x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Flatten()(x)
    
    x = layers.Dropout(0.5)(x)
    
    # Softmax Activation for multiclass classification
    outputs = layers.Dense(196, activation="softmax")(x)
    
    # Instantiate the model
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    # Compile the model
    model.compile(loss = SparseCategoricalCrossentropy(),
                  optimizer=optimizer,
                  metrics=metrics)
    
    return model

In [ ]:
# ===============================================================================================================
# This function takes as input a tensorflow dataset containing test data, and either a trained model
# or the path to where a trained model is located. 
#
# The function then evaluates the model using the test data and returns the associated test metrics.
# ===============================================================================================================
def test_convnet_classifier(test_ds, model=None, model_path=None):
    
    if model is not None:
        
        test_loss, test_acc = model.evaluate(test_ds)
        print("\n========================== Model Test Results ===============================")
        print(f"Test Accuracy: {test_acc}")
        print(f"Test Loss: {test_loss}")
        print("=============================================================================\n")
        
    elif model_path is not None:
        
        model = keras.models.load_model(model_path)
        test_loss, test_acc = model.evaluate(test_dataset)
        print("\n========================== Model Test Results ===============================")
        print(f"Test Accuracy: {test_acc}")
        print(f"Test Loss: {test_loss}")
        print("=============================================================================\n")
        
    else:
        print("\n========================== Error ===============================")
        print("Must pass either a trained model or a path to a trained model file.")
        print("Cannot have both model and model_path = None")
        print("=============================================================================\n")
        return -1
    
    return test_loss, test_acc, model

In [ ]:
# ===============================================================================================================
# This function used to establish callbacks and train the convolutional neural network.
#
# Callbacks are utilized for model checkpoints (save the current best model as determined by val loss), EarlyStopping
# stopping to discontinue training if val_loss has not decreased for 15 iterations, and a learning rate schedule.
# ===============================================================================================================
def train_convnet_classifier(model, train_ds, val_ds, epochs=20, model_save_path=None):
    
    # Callbacks for saving the best model, stopping training when improvements are no longer being made
    # and to decrease the learning rate at specified intervals. 
    callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,
                                                 save_best_only=True,
                                                 monitor="val_loss",
                                                 verbose=1),
                 keras.callbacks.EarlyStopping(monitor="val_loss",
                                               patience=15,
                                               verbose=1),
                 keras.callbacks.LearningRateScheduler(learning_rate_scheduler)]
    
    history = model.fit(train_ds,
                        epochs=epochs,
                        validation_data=val_ds,
                        callbacks=callbacks)
    
    # Save the model history to a .csv
    try:
        
        # Path to save the model history
        history_save_path = model_save_path.split(".")[0] + "_HISTORY.csv"
        
        # Use pandas to save the models history attribute to .csv
        df = pd.DataFrame(history.history)
        df.to_csv(history_save_path, index=False)
        
        # The model checkpoint only saves the "best model", which is not necessarily the final model. This section
        # also saves the final model as it existed after the last epoch of training. This is helpful if it is ever
        # desired to continue training at that point.
        final_save_path = model_save_path.split(".")[0] + "_FINAL_SAVE.keras"
        save_model(model=model, filepath=final_save_path, overwrite=True, include_optimizer=True, save_format='tf')
        
    except:
        print("Couldn't save history or final model!")
    
    return history

In [ ]:
# ===============================================================================================================
# This function uses all the other functions defined above to drive the entire model training process.
# The full process implemented by this function is as follows:
#
# 1. Instantiate and compile the model using the build_convnet_classifier function.
# 2. Generate a uniue filepath to save the best model found during training.
# 3. Train the model and save the history attribute after training.
# 4. Evaluate the best model on the test data.
# ===============================================================================================================
def build_and_train_convnet(train_ds, val_ds, test_ds = None, input_shape=(520, 520, 3), optimizer='adam', metrics=['accuracy'],
                            epochs=20, batch_size=32, lr = 0.001, model_name = 'conv1'):
    
    
    # Build and compile the model
    model = build_convnet_classifier(input_shape=input_shape, optimizer=optimizer, learning_rate = lr, metrics=metrics)
    
    # Display the models summary.
    print(model.summary())
    
    # Get the filepath where the model should be saved.
    model_save_path =  get_model_save_path(optimizer=optimizer, lr=lr, epochs=epochs, batch_size=batch_size, model_name=model_name)
    
    # Display the save filepath
    print(f"Model Checkpoint Save Path: {model_save_path}")
    
    # Fit the neural network. 
    training_history = train_convnet_classifier(model, train_ds, val_ds, epochs=epochs, model_save_path=model_save_path)
    
    # After training, evaluate the best model on the test data.
    if test_ds is not None:
        test_loss, test_acc, best_model = test_convnet_classifier(test_ds, model=None, model_path=model_save_path)
    
    return training_history, best_model

In [ ]:
'''
history, model = build_and_train_convnet(train_ds=train_dataset,
                                         val_ds = val_dataset,
                                         test_ds = test_dataset,
                                         input_shape = (520, 520, 3),
                                         optimizer = 'adam',
                                         metrics=['accuracy'],
                                         epochs=100,
                                         batch_size=32,
                                         lr = 0.0005)''';

# Second Architecture Type

Define a second convnet architecture with one slight difference. Adding another dense layer after the convolutional layers, and prior to the output layer.

In [ ]:
# ===============================================================================================================
# This function is identical to the build_convnet_classifier function shown above, with the addition of 
# a second dense layer in the models output classifier.
# ===============================================================================================================
def build_convnet_classifier_arch2(input_shape, optimizer, learning_rate, metrics):

    optimizer = get_optimizer(optimizer, learning_rate)
    
    # Create a layer that is a set of data augmentations.
    data_augmentation = keras.Sequential(
        [
            layers.experimental.preprocessing.RandomFlip("horizontal"),
            layers.experimental.preprocessing.RandomRotation(0.1),
            layers.experimental.preprocessing.RandomZoom(0.2),
        ]
    )
    
    # Input layer
    inputs = keras.Input(shape=input_shape)
    
    # Perform data augmentation
    x = data_augmentation(inputs)
    
    # Recale the pixel values from the 1-255 range to a 0-1 range
    x = layers.experimental.preprocessing.Rescaling(1./255)(x)
    
    x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    
    x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    
    x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    
    x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    
    x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Flatten()(x)
    
    # Begining of the Dense classifier
    x = layers.Dense(256, activation = 'relu')(x)
    
    x = layers.Dropout(0.5)(x)
    
    # Softmax Activation for binary classification
    outputs = layers.Dense(196, activation="softmax")(x)
    
    # Instantiate the model
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    # Compile the model
    model.compile(loss = SparseCategoricalCrossentropy(),
                  optimizer=optimizer,
                  metrics=metrics)
    
    return model

In [1]:
# ===============================================================================================================
# This function drives the entire process of instantiating, training, and evaluating the second convolutional
# neural network architecture. 
# ===============================================================================================================
def build_and_train_convnet_arch2(train_ds, val_ds, test_ds = None, input_shape=(520, 520, 3), optimizer='adam', metrics=['accuracy'],
                                  epochs=20, batch_size=32, lr = 0.001, model_name = 'conv2'):
    

    # Build and compile the model.
    model = build_convnet_classifier_arch2(input_shape=input_shape, optimizer=optimizer, learning_rate = lr, metrics=metrics)
    
    # Print the model summary.
    print(model.summary())
    
    # Get the path to save the trained model to.
    model_save_path =  get_model_save_path(optimizer=optimizer, lr=lr, epochs=epochs, batch_size=batch_size, model_name=model_name)
    
    # Print the models save path.
    print(f"Model Checkpoint Save Path: {model_save_path}")
    
    # Train the model
    training_history = train_convnet_classifier(model, train_ds, val_ds, epochs=epochs, model_save_path=model_save_path)
    
    # Evaluate the best model on the test data.
    if test_ds is not None:
        
        test_loss, test_acc, best_model = test_convnet_classifier(test_ds, model=None, model_path=model_save_path)
    
    return training_history, best_model

In [ ]:
# Call the function shown above to train the second convnet architecture, save the training history and the
# best model, and evaluate the best model on test data.
history2, model2 = build_and_train_convnet_arch2(train_ds=train_dataset,
                                               val_ds = val_dataset,
                                               test_ds = test_dataset,
                                               input_shape = (520, 520, 3),
                                               optimizer = 'adam',
                                               metrics=['accuracy'],
                                               epochs=100,
                                               batch_size=32,
                                               lr = 0.0005)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 520, 520, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 520, 520, 3)       0         
_________________________________________________________________
rescaling (Rescaling)        (None, 520, 520, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 518, 518, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 259, 259, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 257, 257, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0     